In [1]:
import pyspark
pyspark.__version__

'3.5.1'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, when
spark = SparkSession.builder.appName('EDA').getOrCreate()

25/09/30 12:43:15 WARN Utils: Your hostname, Pongsakorns-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.10.116.201 instead (on interface en0)
25/09/30 12:43:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/30 12:43:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
df_pyspark = spark.read.csv("sales_data_set.csv", header=True, inferSchema=True)
print("[df_pyspark]")
print(f"{'Type:'.ljust(30)} {type(df_pyspark)}")
print(f"{'Total rows:'.ljust(30)} {df_pyspark.count()}")
print(f"{'Total rows without all null:'.ljust(30)} {df_pyspark.na.drop(how='all').count()}")
print(f"{'Columns:'.ljust(30)} {df_pyspark.columns}")

[df_pyspark]
Type:                          <class 'pyspark.sql.dataframe.DataFrame'>
Total rows:                    421570
Total rows without all null:   421570
Columns:                       ['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday']


In [ ]:
# show top 5 rows
print(df_pyspark.show(5))
print("-"*50)

# show schema
print(df_pyspark.printSchema())
print("-"*50)

# show few columns
print(df_pyspark.select(['Store', 'Date','Weekly_Sales']).show(10))

+-----+----+----------+------------+---------+
|Store|Dept|      Date|Weekly_Sales|IsHoliday|
+-----+----+----------+------------+---------+
|    1|   1|05/02/2010|     24924.5|    false|
|    1|   1|12/02/2010|    46039.49|     true|
|    1|   1|19/02/2010|    41595.55|    false|
|    1|   1|26/02/2010|    19403.54|    false|
|    1|   1|05/03/2010|     21827.9|    false|
+-----+----+----------+------------+---------+
only showing top 5 rows

None
--------------------------------------------------
root
 |-- Store: integer (nullable = true)
 |-- Dept: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- Weekly_Sales: double (nullable = true)
 |-- IsHoliday: boolean (nullable = true)

None
--------------------------------------------------
+-----+----------+------------+
|Store|      Date|Weekly_Sales|
+-----+----------+------------+
|    1|05/02/2010|     24924.5|
|    1|12/02/2010|    46039.49|
|    1|19/02/2010|    41595.55|
|    1|26/02/2010|    19403.54|
|    1|05/03/

25/09/30 14:26:30 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1016855 ms exceeds timeout 120000 ms
25/09/30 14:26:30 WARN SparkContext: Killing executors is not supported by current scheduler.
25/09/30 14:43:48 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 

In [36]:
# describe
df_pyspark.describe().show()

+-------+------------------+------------------+----------+------------------+
|summary|             Store|              Dept|      Date|      Weekly_Sales|
+-------+------------------+------------------+----------+------------------+
|  count|            421570|            421570|    421570|            421570|
|   mean|22.200545579619043| 44.26031738501317|      NULL|15981.258123467534|
| stddev| 12.78529738990308|30.492054015786014|      NULL|22711.183519163187|
|    min|                 1|                 1|01/04/2011|          -4988.94|
|    max|                45|                99|31/12/2010|         693099.36|
+-------+------------------+------------------+----------+------------------+



In [56]:
# Add columns
df_pyspark = df_pyspark.withColumn("Sales_Category", when(col("Weekly_Sales") < 30000, "Low")
                                                  .when((col("Weekly_Sales") >= 30000) & (col("Weekly_Sales") < 50000), "Medium")
                                                  .otherwise("High"))

print("Low Sale performece: ".ljust(30),df_pyspark.filter(col("Sales_Category")=="Low").count(), f"({df_pyspark.filter(col("Sales_Category")=="Low").count()/df_pyspark.count():.2%})")
print("Medium Sale performece: ".ljust(30),df_pyspark.filter(col("Sales_Category")=="Medium").count(), f"({df_pyspark.filter(col("Sales_Category")=="Medium").count()/df_pyspark.count():.2%})")
print("High Sale performece: ".ljust(30),df_pyspark.filter(col("Sales_Category")=="High").count(), f"({df_pyspark.filter(col("Sales_Category")=="High").count()/df_pyspark.count():.2%})")

Low Sale performece:           351950 (83.49%)
Medium Sale performece:        37436 (8.88%)
High Sale performece:          32184 (7.63%)


25/09/28 04:20:08 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 932775 ms exceeds timeout 120000 ms
25/09/28 04:20:08 WARN SparkContext: Killing executors is not supported by current scheduler.
25/09/28 04:20:08 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o